***GENERATED CODE FOR teslaaccidentpredictionml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run teslaaccidentpredictionmlHooks.ipynb
try:
	#sourcePreExecutionHook()

	cleanedautofedata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-4444726e-45bb-4194-82cf-85f6639c0eed-c000.csv', 'filename': '1707681841CleanedautoFEData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/Tesla Models Accidents Impact Prediction/CleanedautoFEData.csv', 'viewFileName': 'CleanedautoFEData.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run teslaaccidentpredictionmlHooks.ipynb
try:
	#transformationPreExecutionHook()

	teslaaccidentpredictionmlautofe = TransformationMain.run(cleanedautofedata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Case #", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "195.69", "stddev": "113.03", "min": "1", "max": "392", "missing": "0"}, "updatedLabel": "Case #"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "2020.78", "stddev": "2.23", "min": "2013", "max": "2024", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deaths", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "1.24", "stddev": "0.58", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "Deaths"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Date _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "154.18", "stddev": "106.09", "min": "0.0", "max": "344.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Date _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Country _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "1.42", "stddev": "3.86", "min": "0.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Country _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "State _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "5.15", "stddev": "8.0", "min": "0.0", "max": "37.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "State _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Description _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "131.1", "stddev": "101.14", "min": "0.0", "max": "317.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Description _stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Tesla driver _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.43", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Tesla driver _stringindex..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Tesla occupant _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.23", "stddev": "0.55", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Tesla occupant _stringind..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Other vehicle _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "0.48", "stddev": "0.73", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Other vehicle _stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Cyclists/ Peds _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.16", "stddev": "0.41", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Cyclists/ Peds _stringind..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TSLA+cycl / peds _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "0.63", "stddev": "0.79", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TSLA+cycl / peds _stringi..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Model _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "0.73", "stddev": "1.16", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Model _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Autopilot claimed _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.13", "stddev": "0.38", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Autopilot claimed _string..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Verified Tesla Autopilot Deaths _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.1", "stddev": "0.34", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Verified Tesla Autopilot ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Source_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "182.74", "stddev": "112.34", "min": "0.0", "max": "377.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Source_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Note _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "0.14", "stddev": "0.98", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Note _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deceased 1 _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "18.93", "stddev": "34.32", "min": "0.0", "max": "121.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Deceased 1 _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deceased 2 _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "1.04", "stddev": "4.33", "min": "0.0", "max": "28.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Deceased 2 _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deceased 3 _stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "390", "mean": "0.05", "stddev": "0.48", "min": "0.0", "max": "6.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Deceased 3 _stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deceased 4 _stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "390", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Deceased 4 _stringindexer"}]}))

	#transformationPostExecutionHook(teslaaccidentpredictionmlautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run teslaaccidentpredictionmlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(teslaaccidentpredictionmlautofe, ["Case #", "Year", "Date _stringindexer", "Country _stringindexer", "State _stringindexer", "Description _stringindexer", "Tesla driver _stringindexer", "Tesla occupant _stringindexer", "Other vehicle _stringindexer", "Cyclists/ Peds _stringindexer", "TSLA+cycl / peds _stringindexer", "Model _stringindexer", "Autopilot claimed _stringindexer", "Verified Tesla Autopilot Deaths _stringindexer", "Source_stringindexer", "Note _stringindexer", "Deceased 1 _stringindexer", "Deceased 2 _stringindexer", "Deceased 3 _stringindexer", "Deceased 4 _stringindexer"], "Deaths")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

